# 1. Imports

In [ ]:
import numpy as np
from math import pi

from matplotlib import pyplot as plt

import seaborn as sn
import pandas as pd

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
from tensorflow.keras.losses import MSE
from tensorflow.keras.losses import categorical_crossentropy as CCE
import tensorflow as tf

In [ ]:
from art.attacks.evasion import *
from art.estimators.classification import TensorFlowV2Classifier

import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# Load properties from another python file
from properties import *

In [ ]:
from random import random, randint

# 2. Load models

In [ ]:
path_model = "ACAS_XU_tf_keras/ACASXU_1_1.h5"

In [ ]:
model_11 = load_model(path_model)

In [ ]:
model_35 = load_model("ACAS_XU_tf_keras/ACASXU_3_5.h5")

In [ ]:
model_11.summary()

In [ ]:
model_11.compile()

In [ ]:
model_35.compile()

In [ ]:
pt = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(1,5)
model_11.predict(pt)

In [ ]:
pt = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(1,5)
model_35.predict(pt)

In [ ]:
ACAS_model = model_11

# 3. Carlini & Wagner = CW

In [ ]:
def generate_adv_sample(model, x0, label, loss_function="MSE", eps=1e-5):
    # transforming into a tensorflow object
    x0_ = tf.cast(x0, tf.float32)
    
    # record our gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our input should be tacked for gradient updates
        tape.watch(x0_)

        # use our model to make predictions on the input and then compute the loss
        pred = model(x0_)
        if loss_function == "CCE":
            np_label = np.array([i==label for i in range(0,5)]).reshape((1,5))
            loss = CCE(np_label, pred)
        elif loss_function == "MSE":
            loss = MSE(label, pred)
        else:
            raise Exception("Unknown loss function '{0}'".format(loss_function))
        
        # calculate the gradients of loss with respect to the input, then compute the sign of the gradient
        gradient = tape.gradient(loss, x0_)
        signedGrad = tf.sign(gradient)

        # construct the image adversary
        adv_sample = (x0_ + (signedGrad * eps)).numpy()

        # return the adversarial sample to the calling function
        return(adv_sample)

# 4. Generate points

In [ ]:
def random_pts(n, prop_dom, main_dom):
    """ prop_dom is a list of domains given by intervals in a numpy 2x5 table. """
    ndom = len(prop_dom)
    ndim = main_dom.shape[1]
    
    x = np.zeros((n,ndim))
    for i in range(n): # generate the i-th point
        choosen_dom = prop_dom[randint(0,ndom-1)] # choose the input property domain for a given prop
        for k in range(ndim): # create a random coord for each dim
            boundaries = main_dom[:,k]
            if choosen_dom[0,k] != None:
                boundaries[0] = choosen_dom[0,k]
            if choosen_dom[1,k] != None:
                boundaries[1] = choosen_dom[1,k]
            x[i,k] = boundaries[0] + random()*(boundaries[1]-boundaries[0])

    return(x)

In [ ]:
def random_main_pts(n, main_dom):
    """ Generate random points in the main domain given by intervals stored in a numpy 2x5 table. """
    ndim = main_dom.shape[1]
    
    x = np.zeros((n,ndim))
    for k in range(ndim): # create a random coord for each dim
        col = main_dom[0,k]*np.ones((n,1)) + np.random.rand(n,1)*(main_dom[1,k] - main_dom[0,k])
        for i in range(n):
            x[i,k] = col[i]

    return(x)

In [ ]:
x_test = random_main_pts(1000, X_dom)

In [ ]:
def normalize(x, x_mean, x_range):
    (n,k) = x.shape
    nx = np.zeros((n,k))
    for i in range(n):
        for j in range(k):
            nx[i,j] = (x[i,j]-x_mean[j])/x_range[j]
    return(nx)

In [ ]:
def check_pts(model, input_pts, IP, OP):
    n = input_pts.shape[0]
    IO_check = np.zeros((n,2)) # 1st column : input checked - 2nd column : output checked
    
    norm_input = normalize(input_pts, X_mean, X_range) # normalize pts
    
    pred_pts = model.predict(norm_input) # make predictions with the model (neural net)
    
    for k in range(n):
        IO_check[k,0] = IP(input_pts[k,:]) # check input (just in case)
        IO_check[k,1] = OP(pred_pts[k,:]) # check output

    return(IO_check, pred_pts)

In [ ]:
rand_inputs = random_pts(1000000, IP2_dom, X_dom)

In [ ]:
Pcheck, pred_pts = check_pts(model_11, rand_inputs, IP1, OP1)

In [ ]:
def find_adverse(input_pts, prop_check):
    n = prop_check.shape[0]
    index = []
    for k in range(n):
        if prop_check[k,0] and not(prop_check[k,1]):
            index.append(k)
    return(input_pts[index,:])

In [ ]:
adv = find_adverse(rand_inputs, Pcheck)
print(adv.shape)

In [ ]:
Pcheck.shape

# 5. Attack the networks : FGSM & CW

In [ ]:
ACAS_model = load_model("ACAS_XU_tf_keras/ACASXU_1_1.h5")

### Define the attacks

In [ ]:
# Classifier object <~> neural network ACAS-Xu ...

ACAS_classifier = TensorFlowV2Classifier(model = ACAS_model,
                                         loss_object = MSE, # MSE or CCE ?...
                                         train_step = None,
                                         nb_classes = 5,
                                         input_shape = (1, 5, 1),
                                         clip_values = (-1.0, 1.0))

In [ ]:
# x_test = 0.8*np.array([ [1.0, 1.0, 1.0, 1.0, 1.0],
#                         [0.6, 0.5, 1.0, 0.3, 1.0] ])

Carlini-Wagner (CW) attack

In [ ]:
attack_CW = CarliniLInfMethod(classifier=ACAS_classifier,
                              initial_const = 1.0, 
                              max_iter = 100, 
                              # targeted=True, 
                              verbose = True)

# x_adv_cw = attack_CW.generate(x_test) #, y=np.array([[0,1,0,0,0]]))
# print(x_adv_cw)

Fast Gradient Sign Method (FGSM) attack

In [ ]:
intensity_fgsm = 0.8

attack_FGSM = FastGradientMethod(estimator = ACAS_classifier,
                                 eps = intensity_fgsm,
                                 verbose = True)

# x_adv_fgsm = attack_FGSM.generate(x_test)
# print(x_adv_fgsm)

### Create random (normalized) points

In [ ]:
original_pts = np.random.rand(1000, 5) # random_main_pts(10, X_dom)

In [ ]:
print(original_pts)

### Apply the attacks

In [ ]:
# CW
adv_pts_cw = attack_CW.generate(original_pts)

In [ ]:
# FGSM
adv_pts_fgsm = attack_FGSM.generate(original_pts)

In [ ]:
print("CW adv points")
print(adv_pts_cw,"\n")

print("FGSM adv points")
print(adv_pts_fgsm)

In [ ]:
# Make predictions with the adversarial points

original_pred = ACAS_model.predict(original_pts)
adv_pred_cw = ACAS_model.predict(adv_pts_cw)
adv_pred_fgsm = ACAS_model.predict(adv_pts_fgsm)

print("Original pred :")
print(original_pred,"\n")

print("CW pred :")
print(adv_pred_cw,"\n")

print("FGSM pred :")
print(adv_pred_fgsm)

In [ ]:
# Get advices from the predictions of the network (prediction = min score)

original_advice = np.argmin(original_pred, axis=1)
cw_advice = np.argmin(adv_pred_cw, axis=1)
fgsm_advice = np.argmin(adv_pred_fgsm, axis=1)

print(original_advice[:20])
print(cw_advice[:20])
print(fgsm_advice[:20])

### Results visualization with confusion matrix

In [ ]:
def confusion_matrix(pred0, pred1, n_classes=5):
    """ Create the n_classes*n_classes confusion matrix of pred1 compared to the original pred0. 
        - cmat[i,j] = nb of pts classified as i at the origin and as j after the attack """
    cmat = np.zeros((n_classes,n_classes)).astype(int)
    n_sample = pred0.shape[0]
    for k in range(n_sample):
        cmat[pred0[k],pred1[k]] += 1
    return(cmat)

In [ ]:
def norm_rows(cmat):
    """ Normalize the lines of the confusion matrix cmat. """
    n = cmat.shape[0]
    ncmat = np.zeros((n,n))
    sum_rows = np.sum(cmat_cw, axis=1)
    for k in range(n):
        s = sum_rows[k]
        if s != 0:
            ncmat[k,:] = cmat[k,:]/sum_rows[k]
    return(ncmat)

In [ ]:
# confusion matrix
cmat_cw = confusion_matrix(original_advice, cw_advice)
print(cmat_cw)

# confusion matrix with normalized lines
ncmat_cw = norm_rows(cmat_cw)
print(ncmat_cw)

In [ ]:
# confusion matrix
cmat_fgsm = confusion_matrix(original_advice, fgsm_advice)
print(cmat_fgsm)

# confusion matrix with normalized lines
ncmat_fgsm = norm_rows(cmat_fgsm)
print(ncmat_fgsm)

In [ ]:
# confusion matrix plot legend
index0 = ["COC","WR","WL","SR","SL"]
index1 = ["adv-COC","adv-WR","adv-WL","adv-SR","adv-SL"]

### Plot confusion matrix for CW attack

In [ ]:
df_cmat = pd.DataFrame(ncmat_cw, index = index0, columns = index1)

plt.figure(figsize = (10,7))
sn.heatmap(df_cmat, annot=True)
plt.title("Confusion matrix - CW - #{0}".format(1000))
plt.show()

### Plot confusion matrix for FGSM attack

In [ ]:
df_cmat = pd.DataFrame(ncmat_fgsm, index = index0, columns = index1)

plt.figure(figsize = (10,7))
sn.heatmap(df_cmat, annot=True)
plt.title("Confusion matrix - FGSM - #{0}".format(1000))
plt.show()